## PIV Pipeline example

In [1]:
import json

import cv2
from matplotlib import pyplot as plt

import river.core.define_roi_masks as drm

In [9]:
from pathlib import Path

image_1_path = Path('0000000001.jpg')
image_2_path = Path('0000000003.jpg')

image_1 = cv2.imread(image_1_path)
image_2 = cv2.imread(image_2_path)


In [17]:
uav_matrix = json.loads(Path('uav_transformation_matrix.json').read_text())

In [18]:
xsections = json.loads(Path('xsections.json').read_text())

In [19]:
height_roi = 5

In [20]:
mask, bbox = drm.create_mask_and_bbox(image_1, xsections, uav_matrix, height_roi)

In [7]:
with open('bbox.json', 'w') as file:
    file.write(json.dumps(bbox))
with open('mask.json', 'w') as file:
    file.write(json.dumps(mask.tolist()))

In [23]:
from river.core.piv_pipeline import run_test

results = run_test(image_1_path, image_2_path, mask, bbox, filter_sub_background=False)

In [22]:
from river.core.piv_pipeline import run_analyze_all

images_location = Path('frames/')
results = run_analyze_all(images_location, mask=mask, bbox=bbox, filter_sub_background=True)

Processing 31 frames...


error: OpenCV(4.10.0) /io/opencv/modules/core/src/arithm.cpp:658: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'arithm_op'


In [24]:
import numpy as np

xtable = np.array(results['x']).reshape(results['shape'])
ytable = np.array(results['y']).reshape(results['shape'])
#Calculte the median velocity field
u_median = np.array(results['u_median']).reshape(results['shape'])
v_median = np.array(results['v_median']).reshape(results['shape'])
#Calculte the velocity field #75
u_75 = np.array(results['u'][20]).reshape(results['shape'])
v_75 = np.array(results['v'][20]).reshape(results['shape'])
fig, ax = plt.subplots(1)
ax.imshow(cv2.imread(image_1_path, cv2.IMREAD_GRAYSCALE))
ax.imshow(mask, alpha=0.2,cmap='grey')
ax.quiver(xtable, ytable, u_median, -v_median, color='red')
ax.quiver(xtable, ytable, u_75, -v_75, color='green')
ax.quiver(xtable, ytable, u_75, -v_75, color='green')

KeyError: 'u_median'